In [ ]:
import keras
from keras.models import Sequential, Model
from keras.layers import Input,Conv2D,Dropout,Dense,Flatten,BatchNormalization
from keras.layers import Conv2DTranspose,UpSampling2D,Reshape,Activation
from keras.optimizers import RMSprop

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
input_images = r"C:\Users\Munna chowhan\apple.npy"

In [ ]:
data = np.load(input_images)

In [ ]:
data = data / 255
data = np.reshape(data,(data.shape[0], 28, 28 , 1))
img_w, img_h = data.shape[1:3]

In [ ]:
data.shape

In [ ]:
_ = plt.imshow(data[4343,:,:,0])

In [ ]:
def build_discriminator(num_of_neurons = 64,drop_out = 0.4):
    
    #input layer:
    input_layer = Input(shape=(img_w,img_h,1))
    
    conv_layer_1 = Conv2D(num_of_neurons * 1, 5,strides= 2, activation="relu", padding="same")(input_layer)
    conv_layer_1 = Dropout(drop_out)(conv_layer_1)
    
    conv_layer_2 = Conv2D(num_of_neurons * 2, 5, strides=2, activation="relu", padding="same")(conv_layer_1)
    conv_layer_2 = Dropout(drop_out)(conv_layer_2)
    
    conv_layer_3 = Conv2D(num_of_neurons * 4, 5,strides= 2, activation="relu", padding="same")(conv_layer_2)
    conv_layer_3 = Dropout(drop_out)(conv_layer_3)
    
    conv_layer_4 = Conv2D(num_of_neurons * 8, 5,strides= 1, activation="relu", padding="same")(conv_layer_3)
    conv_layer_4 = Flatten() (Dropout(drop_out)(conv_layer_4))
    
    #output layer:
    output_layer = Dense(units=1,activation="sigmoid")(conv_layer_4)
    
    model = Model(input_layer, output_layer)
    
    model.summary()
    
    return model
    

In [ ]:
discriminator = build_discriminator()

In [ ]:
discriminator.compile(optimizer=RMSprop(lr = 0.0008,decay=6e-8 ,clipvalue = 1.0 ), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
def build_generator(laten_space_dim = 100, filters = 64, drop_out = 0.4):
    
    #input layer:
    input_layer = Input((laten_space_dim,))
    
    #Dense layer:
    dense_layer = Dense(units=7*7*filters)(input_layer)
    dense_layer = BatchNormalization(momentum=0.9)(dense_layer)
    dense_layer = Activation(activation="relu")(dense_layer)
    dense_layer = Reshape(target_shape=(7,7,64))(dense_layer)
    dense_layer = Dropout(drop_out)(dense_layer)
    
    #Deconv layer:
    # upsample is used to increase the size from (7,7) -- >(28,28)
    de_conv_layer_1 = UpSampling2D()(dense_layer)
    de_conv_layer_1 = Conv2DTranspose(filters=int(filters/2), kernel_size=5, padding="same", activation=None)(de_conv_layer_1)
    de_conv_layer_1 = BatchNormalization(momentum=0.9)(de_conv_layer_1)
    de_conv_layer_1 = Activation(activation="relu")(de_conv_layer_1)
    
    de_conv_layer_2 = UpSampling2D()(de_conv_layer_1)
    de_conv_layer_2 = Conv2DTranspose(filters=int(filters/4), kernel_size=5, padding="same", activation=None)(de_conv_layer_2)
    de_conv_layer_2 = BatchNormalization(momentum=0.9)(de_conv_layer_2)
    de_conv_layer_2 = Activation(activation="relu")(de_conv_layer_2)
    

    de_conv_layer_3 = Conv2DTranspose(filters=int(filters/8), kernel_size=5, padding="same", activation=None)(de_conv_layer_2)
    de_conv_layer_3 = BatchNormalization(momentum=0.9)(de_conv_layer_3)
    de_conv_layer_3 = Activation(activation="relu")(de_conv_layer_3)
    
    #combine all the filters to make it in to one single image 
    output_layer = Conv2D(filters=1,kernel_size=5, padding="same",activation="sigmoid")(de_conv_layer_3)
    
    model = Model(input_layer, output_layer)
    model.summary()
    
    return model

In [ ]:
generator = build_generator()

In [ ]:
def build_adversarial(laten_space_dim = 100):
    
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(loss="binary_crossentropy", optimizer = RMSprop(lr = 0.0004, decay = 3e-8, clipvalue =1.0), metrics = ["accuracy"])
    model.summary()
    return model

In [ ]:
adversarial_network = build_adversarial()

In [ ]:
def make_trainable(network, value):
    network.trainable = value
    for layer in network.layers:
        layer.trainable = value
   

In [ ]:
def train(epochs = 2000, batch_size = 128):
    
    
    d_metrics = []
    a_metrics = []
    
    running_d_loss = 0
    running_d_acc = 0
    
    running_a_loss = 0
    running_a_acc = 0
    
    for epoch in range(epochs):
        print("Epoch: {} ".format(epoch + 1))
    
        # take real and fake images to feed in to discriminator
        real_imgs = np.reshape(data[np.random.choice(data.shape[0],size= batch_size,replace=False)], (batch_size,28,28,1))
        fake_imgs = generator.predict(np.random.uniform(-1.0,1.0,size=(batch_size, 100)))#predict 128 images from 100 dim latent space
        #(random noise)
        
        #concatenate both the images
        x = np.concatenate((real_imgs, fake_imgs))
        #2d array containing ones
        y = np.ones([batch_size*2,1])
        # label second half of y to have 0 
        y[batch_size:,:] = 0
        
        # make the discriminator network trainable to first classify fake and real images
        make_trainable(discriminator, True)
         # we have to compile the after changing the trainable parameter
        discriminator.compile(optimizer=RMSprop(lr = 0.0008,decay=6e-8 ,clipvalue = 1.0 ), loss="binary_crossentropy", metrics=["accuracy"])

        #train on that single batch and get metrics
        d_metrics.append(discriminator.train_on_batch(x, y))
        # get loss 
        running_d_loss += d_metrics[-1][0]
        # get acc
        running_d_acc += d_metrics[-1][1]
        # make the discriminator freeze to train generator
        make_trainable(discriminator, False)
         # we have to re - compile the after changing the trainable parameter
        discriminator.compile(optimizer=RMSprop(lr = 0.0008,decay=6e-8 ,clipvalue = 1.0 ), loss="binary_crossentropy", metrics=["accuracy"])

        # feed noise into generator
        noise = np.random.uniform(-1.0,1.0,size=(batch_size, 100))
        # feeding the noise images with labels as 1 to trick the discriminator and generate real looking images by updating the loss on noise images compared to the real images

        y = np.ones([batch_size,1])
        # train on the batch and get metrics
        a_metrics.append(adversarial_network.train_on_batch(noise, y))
        # get loss
        running_a_loss += a_metrics[-1][0]
        #get accuracy
        running_a_acc += a_metrics[-1][1]
        
        if (epoch + 1) % 500 == 0:
            # displaying log metrics
            print("Epoch: {} ".format(epoch + 1))
            log_msg = "%d: [D loss: %f, acc: %f]" %(epoch,running_d_loss/epoch, running_d_acc / epoch)
            log_msg = "%s: [A loss: %f, acc: %f]" %(log_msg, running_a_loss/epoch, running_a_acc/ epoch)
            print(log_msg)
            
            # randomly generate 16 noise samples from 100 dim latent space
            noise = np.random.uniform(-1.0,1.0,size=(16, 100))
            #predict those 16 noise data
            predicted_imgs = generator.predict(noise)
            
            #plotting 
            plt.figure(figsize=(8,8))
            # loop through all the images
            for img in range(predicted_imgs.shape[0]):
                
                plt.subplot(4,4, img + 1)
                plt.imshow(predicted_imgs[img,:,:,0], cmap="gray")
                plt.axis("off")
            
            plt.tight_layout()
            plt.show()
            
    return a_metrics, d_metrics

In [ ]:
a_metrics, d_metrics = train(epochs=10)

In [ ]:
loss = pd.DataFrame( {"Discriminator Loss": [metric[0] for metric in d_metrics],
                      "Generator Loss": [metric[0] for metric in a_metrics]}).plot(title="Training loss",logy=True)
loss.set_xlabel("Epochs")
loss.set_ylabel("Loss");

In [ ]:
accuracy = pd.DataFrame( {"Discriminator Accuracy": [metric[1] for metric in d_metrics],
                      "Generator Accuracy": [metric[1] for metric in a_metrics]}).plot(title="Training accuracy")
accuracy.set_xlabel("Epochs")
accuracy.set_ylabel("Accuracy");